## NLPのファインチューニングをpythorchで実行する際のtutorialを行う

In [1]:
cd ..

/home/is/akiyoshi-n/my-project


### データ作成のためのコード

In [2]:
from transformers import AutoTokenizer
# 東北大バートversion2モデルを使用

MODEL_NAME = 'cl-tohoku/bert-base-japanese-v2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [3]:
encoded_input = tokenizer('私は先生です．生徒に数学を教えています．')
print(encoded_input)

{'input_ids': [2, 3946, 897, 14385, 12461, 28, 12203, 893, 14167, 932, 13507, 888, 854, 12343, 28, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [4]:
tokenizer.decode(encoded_input["input_ids"])

'[CLS] 私 は 先生 です. 生徒 に 数学 を 教え て い ます. [SEP]'

In [5]:
batch_sentences = [
    '私は先生です．生徒に数学を教えています．',
    '私は生徒です．先生に数学を教えてもらっています．',
    '私は校長です．先生や生徒に生き方を教えています．',
]
# padding: 最大長に満たない文はパディングを行う
# truncation: 最大長を超える文は切り捨てる
# return_tensors: 出力をPyTorchのテンソルにする
encoded_inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    2,  3946,   897, 14385, 12461,    28, 12203,   893, 14167,   932,
         13507,   888,   854, 12343,    28,     3,     0,     0,     0],
        [    2,  3946,   897, 12203, 12461,    28, 14385,   893, 14167,   932,
         13507,   888, 16744,   888,   854, 12343,    28,     3,     0],
        [    2,  3946,   897, 15681, 12461,    28, 14385,   918, 12203,   893,
         12269,  2706,   932, 13507,   888,   854, 12343,    28,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


### データセット作成

In [3]:
from transformers import AutoTokenizer
MODEL_NAME = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [4]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")
dataset['train'][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [5]:
# examples['text']を入力としてトークナイズする関数
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [6]:
print(type(dataset))
print(type(dataset['train']))
print(type(dataset['train']['label']))
print(dataset['train']['label'][100])
print(type(dataset['train']['text']))
print(dataset['train']['text'][100])

<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'list'>
0
<class 'list'>
My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\nThe cashier took my friends's order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid's meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \"serving off their orders\" when they didn't have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\nThe manager was rude when giving me my order. She didn't make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was ge

In [16]:
# tokenize_functionを使ってデータセットをトークナイズする
# tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
# pickleで読み込み
import pickle
with open('tokenized_dataset.pickle', 'rb') as f:
    tokenized_dataset = pickle.load(f)

次にtokenized_datasetを手動で後処理して学習ように準備．

In [4]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [5]:
# この中から少ないデータセットを作成する
small_train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset['test'].shuffle(seed=42).select(range(1000))

In [10]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [13]:
type(small_train_dataset)

datasets.arrow_dataset.Dataset

### 分類モデル

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


事前学習された重みの一部が使用されておらず、一部の重みがランダムに初期化されているという警告が表示されます。ご心配なく、これは完全に正常です！BERTモデルの事前に訓練されたヘッドは破棄され、ランダムに初期化された分類ヘッドに置き換えられます。この新しいモデル・ヘッドは、事前訓練されたモデルの知識を移行させながら、シーケンス分類タスクで微調整されます。

#### 学習ハイパーパラメータ
TrainingArguments: このクラスには調整可能な全てのハイパーパラメータとさまざまな学習optionを有効にするためのフラグが含まれている．

In [8]:
#### 学習ハイパーパラメータ
from transformers import TrainingArguments
# 学習のチェックポイントを保存するディレクトリ
training_args = TrainingArguments(output_dir='test_trainer')

In [9]:
import numpy as np
import evaluate
metric = evaluate.load('accuracy')

評価に関するサイト
https://huggingface.co/docs/evaluate/a_quick_tour

予測値の精度を計算するために、metricでcomputeを呼び出す．しかし，予測値をcomputeに渡す前に，logitsを予測値に変換する必要があります（すべてのTransformersモデルはlogitsを返すことを覚えておく）

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # np.argmaxで最大値をとるインデックスを取得
    # axis=-1で最後の次元方向にargmaxをとる．基本クラス毎
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Fine-tuning中に評価値を見たい場合は学習引数にevaluation_strategyパラメータを指定し，各エポック終了時に評価値を見れるようにする

In [11]:
training_args = TrainingArguments(output_dir='test_trainer', evaluation_strategy='epoch')

### Trainerオブジェクト作成

In [12]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

## 感情分析例

In [13]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating unsupervised split: 0 examples [00:00, ? examples/s]

In [12]:
type(imdb)

NameError: name 'imdb' is not defined

In [14]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [35]:
type(imdb['train']['label'])

list

In [34]:
imdb['train']['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [16]:
from transformers import AutoTokenizer
MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [18]:
# batched=Trueで複数のデータをまとめて処理する
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

### DataCollatorWithPadding
これを使用し，例文のバッチを作成する．データ全体を最大長にパディングするのではなく，照合時にバッチ内の最長長に動的にパディングする方が効率的．

In [19]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### 評価

In [20]:
import evaluate
accuracy = evaluate.load('accuracy')

In [21]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

### 訓練

In [22]:
# モデルの学習を開始する前にid2labelとlabel2idで予想されるidとラベルのマップを作成
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [23]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [27]:
trainer.train()

/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: Caught OutOfMemoryError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/transformers/models/distilbert/modeling_distilbert.py", line 1000, in forward
    distilbert_output = self.distilbert(
                        ^^^^^^^^^^^^^^^^
  File "/home/is/akiyoshi-n/my-project/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/is/akiyoshi-

In [28]:
import subprocess
import json

def get_gpu_info():
    try:
        # nvidia-smiコマンドを実行し、出力をJSON形式で取得
        smi_output = subprocess.check_output(['nvidia-smi', '--query-gpu=index,name,memory.total,memory.free,memory.used', '--format=csv,noheader,nounits'])
        smi_output = smi_output.decode('utf-8')

        # 出力を解析して各GPUの情報をリストに格納
        gpu_info = []
        for line in smi_output.strip().split('\n'):
            index, name, total, free, used = line.split(', ')
            gpu_info.append({
                'index': index,
                'name': name,
                'memory_total': int(total),
                'memory_free': int(free),
                'memory_used': int(used)
            })

        return gpu_info

    except Exception as e:
        print(f"Error: {e}")
        return []

def main():
    # GPU情報を取得
    gpu_info = get_gpu_info()

    # GPU情報を表示
    if gpu_info:
        for gpu in gpu_info:
            print(f"GPU {gpu['index']}: {gpu['name']}")
            print(f"  Total Memory: {gpu['memory_total']} MiB")
            print(f"  Free Memory: {gpu['memory_free']} MiB")
            print(f"  Used Memory: {gpu['memory_used']} MiB\n")
    else:
        print("No GPU found or unable to retrieve GPU information.")

if __name__ == "__main__":
    main()


GPU 0: Quadro P6000
  Total Memory: 24576 MiB
  Free Memory: 18921 MiB
  Used Memory: 5522 MiB

GPU 1: Quadro RTX 8000
  Total Memory: 49152 MiB
  Free Memory: 14957 MiB
  Used Memory: 33635 MiB

GPU 2: Quadro RTX 8000
  Total Memory: 49152 MiB
  Free Memory: 5 MiB
  Used Memory: 48585 MiB

